In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/dlnlpAssign1/

/content/drive/MyDrive/dlnlpAssign1


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.utils.data as data_utils

import string
import numpy as np

In [4]:
filename1 = 'Train.pos'
filename2 = 'Train.neg'
filenameTest = 'TestData'

with open(filename1, encoding = "ISO-8859-1") as my_file:
    posSentences = my_file.readlines()
# posSentences = posSentences[:100]
lenPos = len(posSentences)
print(lenPos)

with open(filename2, encoding = "ISO-8859-1") as my_file:
    negSentences = my_file.readlines()
# negSentences = negSentences[:100]
lenNeg = len(negSentences)
print(lenNeg)

with open(filenameTest, encoding = "ISO-8859-1") as my_file:
    testSentences = my_file.readlines()
# negSentences = negSentences[:100]
lenTest = len(testSentences)
print(lenTest)

5000
5000
662


In [5]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

gloveFile = "glove.6B.200d.txt"
word_to_vec_map = read_glove_vector(gloveFile)
# print(word_to_vec_map['rock'])

In [6]:
from gensim.models import Word2Vec, KeyedVectors
w2vmodel = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [7]:
dataset = []
table = str.maketrans('', '', string.punctuation)
# posSentences = posSentences[:6]
# negSentences = negSentences[:6]

for sentence in posSentences:
    sentence = sentence.translate(table)
    words = sentence.strip().split()
    # stripped = [w.translate(table) for w in words]
    dataset.append(words)

for sentence in negSentences:
    sentence = sentence.translate(table)
    words = sentence.strip().split()
    # stripped = [w.translate(table) for w in words]
    dataset.append(words)

testDataset = []
for sentence in testSentences:
    sentence = sentence.translate(table)
    words = sentence.strip().split()
    # stripped = [w.translate(table) for w in words]
    testDataset.append(words)

In [8]:
vecDataset = []
# i = 0
for lo_words in dataset:
  cnt = len(lo_words)
  
  lo_vecs_glove = [word_to_vec_map['unk'] if x not in word_to_vec_map.keys() else word_to_vec_map[x] for x in lo_words]
  sum_vec_glove = sum(lo_vecs_glove)

  lo_vecs_w2v = [w2vmodel['unk'] if x not in w2vmodel.wv.vocab else w2vmodel[x] for x in lo_words]
  sum_vec_w2v = sum(lo_vecs_w2v)
  
  sum_vec = np.append(sum_vec_glove,sum_vec_w2v)
  vecDataset.append(sum_vec/cnt)

testVecDataset = []
for lo_words in testDataset:
  cnt = len(lo_words)
  lo_vecs_glove = [word_to_vec_map['unk'] if x not in word_to_vec_map.keys() else word_to_vec_map[x] for x in lo_words]
  sum_vec_glove = sum(lo_vecs_glove)
  lo_vecs_w2v = [w2vmodel['unk'] if x not in w2vmodel.wv.vocab else w2vmodel[x] for x in lo_words]
  sum_vec_w2v = sum(lo_vecs_w2v)
  sum_vec = np.append(sum_vec_glove,sum_vec_w2v)
  testVecDataset.append(sum_vec/cnt)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [9]:
x_data = torch.tensor(vecDataset, dtype=torch.float)
print(x_data.size())

x_data_test = torch.tensor(testVecDataset, dtype=torch.float)
print(x_data_test.size())

torch.Size([10000, 500])
torch.Size([662, 500])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [10]:
shape = (lenPos,1)
y1 = torch.ones(shape)
y0 = torch.zeros(shape)
y = torch.cat((y1,y0))
print(y.size())

shape1 = (331,1)
y1test = torch.ones(shape1)
y0test = torch.zeros(shape1)
y_test = torch.cat((y1test,y0test))
print(y_test.size())

torch.Size([10000, 1])
torch.Size([662, 1])


In [11]:
full_dataset = data_utils.TensorDataset(x_data, y)
train_size = int(0.75 * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])
train_loader = data_utils.DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = data_utils.DataLoader(validation_dataset, batch_size=64, shuffle=True)

testDataset = data_utils.TensorDataset(x_data_test, y_test)
test_loader = data_utils.DataLoader(testDataset, batch_size=64, shuffle=True)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = nn.Linear(500, 500)  
    self.hid2 = nn.Linear(500, 300)
    self.hid3 = nn.Linear(300, 100)
    self.oupt = nn.Linear(100, 1)
    self.dropout = nn.Dropout(0.5)

  def forward(self, x):
    z = torch.relu(self.hid1(x))
    z = self.dropout(z) 
    z = torch.relu(self.hid2(z))
    z = self.dropout(z)
    z = torch.relu(self.hid3(z))
    z = self.dropout(z)
    z = torch.sigmoid(self.oupt(z))
    return z

In [14]:
model = Net().to(device)

In [15]:

# average vectors over sentence and convert to tensor
# use dataloader
# train then - refer to official site

In [16]:
# print(word_to_vec_map['unk'])

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def validation_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    validation_loss, correct, incorrect = 0, 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            validation_loss += loss_fn(pred, y).item()
            correct += ((pred>=0.5) == y).type(torch.float).sum().item()
            incorrect += ((pred>=0.5) != y).type(torch.float).sum().item()

    validation_loss /= num_batches
    correct /= size
    incorrect /= size
    print(f"validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {validation_loss:>8f} \n")
    print(f"Incorrect percentage : \n Accuracy: {(100*incorrect):>0.1f}%\n")
    return validation_loss

In [19]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    validation_loss, correct, incorrect = 0, 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            validation_loss += loss_fn(pred, y).item()
            correct += ((pred>=0.5) == y).type(torch.float).sum().item()
            incorrect += ((pred>=0.5) != y).type(torch.float).sum().item()

    validation_loss /= num_batches
    correct /= size
    incorrect /= size
    print(f"Test Stats: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {validation_loss:>8f} \n")
    print(f"Incorrect percentage : \n Accuracy: {(100*incorrect):>0.1f}%\n")

In [20]:
def getTrainLoseAndError(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct, incorrect = 0, 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            train_loss += loss_fn(pred, y).item()
            correct += ((pred>=0.5) == y).type(torch.float).sum().item()
            incorrect += ((pred>=0.5) != y).type(torch.float).sum().item()

    train_loss /= num_batches
    correct /= size
    incorrect /= size
    print(f"Train Error : \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    print(f"Incorrect percentage : \n Accuracy: {(100*incorrect):>0.1f}%\n")


In [21]:
learning_rate = 1e-3
batch_size = 64
epochs = 20
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #try adagrad too
print("In the begining :")
getTrainLoseAndError(train_loader, model, loss_fn)
validation_loop(validation_loader, model, loss_fn)
print("------------------")
vLossMin = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    vLoss = validation_loop(validation_loader, model, loss_fn)
    if(vLossMin>vLoss):
      torch.save(model, "modelMinLoss.p")
      vLossMin = vLoss

print("Done!")

print("In the End :")
getTrainLoseAndError(train_loader, model, loss_fn)
validation_loop(validation_loader, model, loss_fn)
print("------------------")

In the begining :
Train Error : 
 Accuracy: 49.9%, Avg loss: 0.693979 

Incorrect percentage : 
 Accuracy: 50.1%

validation Error: 
 Accuracy: 50.0%, Avg loss: 0.694338 

Incorrect percentage : 
 Accuracy: 50.0%

------------------
Epoch 1
-------------------------------
loss: 0.690249  [    0/ 7500]
loss: 0.495395  [ 3200/ 7500]
loss: 0.501509  [ 6400/ 7500]
validation Error: 
 Accuracy: 74.4%, Avg loss: 0.525056 

Incorrect percentage : 
 Accuracy: 25.6%

Epoch 2
-------------------------------
loss: 0.470821  [    0/ 7500]
loss: 0.515739  [ 3200/ 7500]
loss: 0.503476  [ 6400/ 7500]
validation Error: 
 Accuracy: 74.7%, Avg loss: 0.501469 

Incorrect percentage : 
 Accuracy: 25.3%

Epoch 3
-------------------------------
loss: 0.512352  [    0/ 7500]
loss: 0.578902  [ 3200/ 7500]
loss: 0.511824  [ 6400/ 7500]
validation Error: 
 Accuracy: 77.0%, Avg loss: 0.483976 

Incorrect percentage : 
 Accuracy: 23.0%

Epoch 4
-------------------------------
loss: 0.419546  [    0/ 7500]
loss: 0

In [24]:
# torch.save(model, "model1.p")
model = torch.load("modelMinLoss.p")
test_loop(test_loader, model, loss_fn)

Test Stats: 
 Accuracy: 76.1%, Avg loss: 0.468983 

Incorrect percentage : 
 Accuracy: 23.9%

